In [1]:
# monte carlo
import pymc3 as pm
# regression
import lmfit
# basic
import math
import random
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [42]:
%%capture
%run Part1_DataPreparation.ipynb
#%run Astronomy.ipynb
%run CoordinateTransformation.ipynb
#%run Part2_Modeling.ipynb

In [3]:
#d=Digi2Pixel("images/GR_QLBII.jpg", marker=quedlII)

In [4]:
#x=list(d[0]['x [Pixel]'])
#y=list(d[0]['y [Pixel]'])

In [5]:
#maxi=len(y)
#ind=random.randint(0,maxi)
#y.pop(ind)
#y;

#### Define replace function 

In [43]:
def replace(tup, x, y):
   tup_list = list(tup)
   for element in tup_list:
      if element == x:
         tup_list[tup_list.index(element)] = y
   new_tuple = tuple(tup_list)
   return new_tuple

In [44]:
def DumpJSON(filename, path):
    with open(path, 'w') as outfile:
        json.dump(filename, outfile)
    return;

# Linear regression

In [45]:
def replace_all(text):
    
    d = { ",": "/", ")(":",", "/ ":"/", "(":"" ,")":""}
    for i in d.items():
        text = text.replace(i[0], i[1])
    return text

In [57]:
def ConvertCoordinates(digi=listobjs[0][0]["marker"]):
    x=[]
    y=[]

    digi_coord=digi.split('mk=')
    digi_coord=digi_coord[1].split(',')
    for i in range(0, len(digi_coord)):
        x.append((float(digi_coord[i].split('/')[0])))
        y.append(1-((float(digi_coord[i].split('/')[1]))))
    return x,y
# FIT I
def Regression(objects, objects1, mod=1, threshold=10, side="left", sigm=0.2, startvalue=0.5):
    azimuth=list()
    digiliblist=list()
    model = lmfit.models.LinearModel()
    for element in list(objects["gates"].keys()):
      
        try:
            elements=objects["gates"][element]["elements"].keys()
            xmean=list()
            ymean=list()
            if mod==3 or mod==4:
                for i in elements:
                    c=nested_lookup(i, objects["gates"][element]["elements"])
                    #print(len(objects["gates"][element]["elements"]))
                    #for side in ["left", "right"]:
                    xleft,yleft = ConvertCoordinates(digi=objects["digilib"]+','.join(nested_lookup("left", c)))
                    xright,yright = ConvertCoordinates(digi=objects["digilib"]+','.join(nested_lookup("right", c)))
                    xmean.append(np.mean([xleft, xright]))
                    ymean.append(np.mean([yleft, yright]))



            # METHOD 3 ------------
            if mod==3:
                x=xmean
                y=ymean
                mu, sigma = 1, sigm # mean and standard deviation
                s = np.random.normal(mu, sigma, 2)
                x.append(startvalue*s[0])
                y.append(startvalue*s[1])
                # required for regression procedure!
                x.append(startvalue*s[0])
                y.append(startvalue*s[1])
            if mod==4:
                x=xmean
                y=ymean
                mu, sigma = 1, sigm # mean and standard deviation
                s = np.random.normal(mu, sigma, 2)
                x.append(startvalue*s[0])
                y.append(startvalue*s[1])
                # required for regression procedure!
                x.append(startvalue*s[0])
                y.append(startvalue*s[1])
                mu, sigma = 1, 0.01 # mean and standard deviation
                for i in range(len(y)-1):
                    s = np.random.normal(mu, sigma, 2)
                    x=replace(x, x[i], x[i]*s[0])
                    y=replace(y, y[i], y[i]*s[1])
               

            fit = model.fit(y,x=x)
            d=fit.params["slope"]
            if abs(d.stderr/d.value)*100 < threshold:
                # convert data to digilib
                y = [1 - i for i in y]
                f=list(zip(x, y))
                f1=''.join(map(str,f))
                final=objects["digilib"]+replace_all(f1)
                digiliblist.append(replace_all(f1))
                # define azimuth
                if objects["gates"][element]["sector"]==1:
                    azim=90-abs(math.degrees(math.atan(fit.values["slope"])))
                    azimuth.append({"digilib": final,"azimuth":round(azim,5), "stderr[%]": abs(d.stderr/d.value)*100,"gate":float(element), "side":side, "#elements":len(objects["gates"][element]["elements"]), "azimuth(michel)":objects1["gate"]["Michel"]})
                if objects["gates"][element]["sector"]==2:
                    azim=90+abs(math.degrees(math.atan(fit.values["slope"])))
                    azimuth.append({"digilib": final,"azimuth":round(azim,5), "stderr[%]": abs(d.stderr/d.value)*100,"gate":float(element), "side":side, "#elements":len(objects["gates"][element]["elements"]), "azimuth(michel)":objects1["gate"]["Michel"]})
                if objects["gates"][element]["sector"]==3:
                    azim=270-abs(math.degrees(math.atan(fit.values["slope"])))
                    azimuth.append({"digilib": final,"azimuth":round(azim,5), "stderr[%]": abs(d.stderr/d.value)*100,"gate":float(element), "side":side, "#elements":len(objects["gates"][element]["elements"]), "azimuth(michel)":objects1["gate"]["Michel"]})
                if objects["gates"][element]["sector"]==4:
                    azim=270+abs(math.degrees(math.atan(fit.values["slope"])))
                    azimuth.append({"digilib": final,"azimuth":round(azim,5), "stderr[%]": abs(d.stderr/d.value)*100,"gate":float(element), "side":side, "#elements":len(objects["gates"][element]["elements"]), "azimuth(michel)":objects1["gate"]["Michel"]})

        except:
            pass
    plt.show()
    try:
        azimuth={"fitted_marker": objects["digilib"]+','.join(digiliblist) , "object":objects["object"],"latitude": np.mean(nested_lookup("lat", objects["referencepoints"])),"longitude": np.mean(nested_lookup("long", objects["referencepoints"])),"mean azimuth": azimuth}
    except:
        pass
        #azimuth={"fitted_marker": "" , "object":objects["object"], "latitude": None,"mean azimuth": azimuth}
    return azimuth

In [60]:
f=Regression(listobjs[0][0], listobjs[0][2], mod=3, side="both");

In [61]:
def Modification(objects="", objects1="", iterat=400, mods=1, threshold=20, side="left", sigma=0.2, startvalue=0.5):
    listReg=list()
    for i in range(iterat):
        listReg.append(Regression(objects, objects1, mod=mods, threshold=threshold, side=side, sigm=sigma, startvalue=startvalue))
    templist=list()
    finallist={}
    try:
        for i in range(len(listReg)):
            templist.append(listReg[i]["mean azimuth"])
        finallist["object"]=listReg[0]["object"]
       
        finallist["latitude"]=listReg[0]["latitude"]
        finallist["longitude"]=listReg[0]["longitude"]
        finallist["mean azimuth"]=templist
        finallist["fitted marker"]=listReg[0]["fitted_marker"]
        
            
    except:
        pass
    return finallist

In [65]:
f=Modification(objects=listobjs[0][0], objects1=listobjs[0][2], iterat=20, mods=3, threshold=255, side="both", sigma=10.2, startvalue=10.5)
f

{'object': 'QuedlinburgI',
 'latitude': 51.813308250000006,
 'longitude': 11.17835425,
 'mean azimuth': [[{'digilib': 'http://repository.edition-topoi.org/digilib/digilib.html?fn=/KGAL/ReposKGAL/KGAL0001/GR_QLBI.jpg&ww=0.7142&wh=0.7142&wy=0.2857&mk=0.36935/0.34220000000000006,0.347075/0.3126,0.33399999999999996/0.29145,0.24925/0.1884,0.190775/0.11914999999999998,159.40580331455845/-166.54806438058878,159.40580331455845/-166.54806438058878',
    'azimuth': 316.35184,
    'stderr[%]': 0.07739599340178478,
    'gate': 1.0,
    'side': 'both',
    '#elements': 5,
    'azimuth(michel)': [[134, 136], [21, 23], [214, 217], [312, 313]]},
   {'digilib': 'http://repository.edition-topoi.org/digilib/digilib.html?fn=/KGAL/ReposKGAL/KGAL0001/GR_QLBI.jpg&ww=0.7142&wh=0.7142&wy=0.2857&mk=0.50915/0.08210000000000006,-27.445968677072923/34.460365028328184,-27.445968677072923/34.460365028328184',
    'azimuth': 320.88328,
    'stderr[%]': 3.5798691731431596e-14,
    'gate': 2.0,
    'side': 'both',
    

### Create data list for "Modeling_Part4"

In [64]:
def Run(mod=3, iteration=10, threshold=10, side="left"):
    datalist=list()
    for element in listobjs:
        datalist.append(Modification(objects=element[0], objects1=element[2], iterat=iteration, mods=mod, threshold=threshold, side=side))
    return datalist

In [ ]:
#%%capture
#e=Run(mod=1, iteration=2, threshold=10)
#e

In [ ]:
#datalist

In [ ]:
#azimuth_values["4.0"]

In [ ]:
#plt.figure(figsize=(16,12))
#plt.hist(azimuth_values["1.0"], bins=12)
#plt.hist(azimuth_values["2.0"], bins=12)
#plt.hist(azimuth_values["3.0"], bins=12)
#plt.hist(azimuth_values["4.0"], bins=12)
#plt.hist(azimuth_values["5.0"], bins=12)

In [ ]:
#df_data=pd.DataFrame(datalist)
#df_data.head()

In [ ]:
#plt.figure(figsize=(18,10))
#plt.title('distributions of orientations in azimuth at horizon for side: left')
#g=sns.swarmplot(x="site", y="mean azimuth",data=df_data)
#plt.xticks(rotation=90);

In [ ]:
#def az2dec(phi,A):
 #   rad_phi=radians(phi)
 #   rad_A=radians(A)
#    a=degrees(asin(cos(rad_phi)*cos(rad_A))) # for alt=0
#    return(round(a,5))
#def hor_to_equ1(x):
#    return(az2dec(x[1],x[2]))

In [ ]:
#df_data["delta"]=df_data.apply(hor_to_equ1,axis=1)
#df_data.head()

In [ ]:
#plt.figure(figsize=(18,10))
#plt.title('distributions of orientations in declination at horizon for side: left')
#g=sns.swarmplot(x="site", y="delta",data=df_data)
#plt.xticks(rotation=90);

In [ ]:
#listobjs[0]

In [ ]:
              
            
            #for side in ["left", "right"]:
            #if side in ["left", "right"]:
            #    # METHOD 1 ------------
            #    if mod==1:
            #        x,y = ConvertCoordinates(digi=objects["digilib"]+','.join(nested_lookup(side, objects["gates"][element]["elements"])))
            #         maxi=len(y)
            #        ind=random.randint(0,maxi) # select index coincidentally
            #        sign=[-1,1][random.randrange(2)] # select sign coincidentally
            #        x=replace(x, x[ind], x[ind]+0.1*x[ind]*sign)
            #        y=replace(y, y[ind], y[ind]+0.1*y[ind]*sign)
                  
                # METHOD 2 ------------
            #    if mod==2:
            #        x,y = ConvertCoordinates(digi=objects["digilib"]+','.join(nested_lookup(side, objects["gates"][element]["elements"])))
            #        ind=random.randint(1,100)
            #        sign=[-1,1][random.randrange(2)] # select sign coincidentally
            #       x.append(0.5+0.15/ind*sign)
            #       y.append(0.5+0.15/ind*sign)
            #        x.append(0.5+0.15/ind*sign)
            #        y.append(0.5+0.15/ind*sign)
                    